### prompts using ChatPromptTemplate

In [291]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant.",),
    MessagesPlaceholder(variable_name="conversation"),
    HumanMessage(content="What's the capital of France?"),
])

history = [
    HumanMessage(content="Hi!"),
    AIMessage(content="Hello! How can I help?")
]

chat_prompt.format_messages(conversation=history)




[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={})]

### prompts using PromptTemplate and FewShotPromptTemplate

In [292]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain.llms import OpenAI

examples = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="{input} -> {output}",
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="{input} ->",
    input_variables=["input"]
)


prompt.pretty_print()

print("--------------------------------")
example_prompt

Hello -> Bonjour

Goodbye -> Au revoir

{input} ->
--------------------------------


PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='{input} -> {output}')

## Tested PydanticOutputParser - (Working)

In [293]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import (
    AIMessage, HumanMessage, SystemMessage
)

In [294]:
max_essay_length = 10000

In [295]:
class ModelResponse(BaseModel):
    """Model response schema."""
    model_name: str = Field(description="Name of the LLM model that you (AI) is using", max_length=20)
    response: str = Field(description="Response from the model, based on the question. Check the maximum limit mentioned in the prompt", max_length=max_essay_length)
    tagline: str = Field(description="response created by Vivek", max_length=50)
    status: int = Field(description="Status of the response, 0 for success and other values for error.", default=0)

In [296]:
class ErrorModelResponse(BaseModel):
    """Error response schema."""
    model_name: str = Field(description="Name of the LLM model used", max_length=20)
    response: str = Field(description="Response from the model, based on the question. Check the maximum limit mentioned in the prompt", default="Error while processing the request")
    tagline: str = Field(description="response created by Vivek", max_length=50)
    status: int = Field(description="Status of the response, 0 for success and other values for error.", default=1)
    

In [297]:
model = init_chat_model(model="gemini-2.0-flash", model_provider="google_genai")
# model = init_chat_model(model="gemini-2.5-pro", model_provider="google_genai")

In [298]:
# Create the output parser
resp_parser = PydanticOutputParser(pydantic_object=ModelResponse)
resp_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the LLM model that you (AI) is using", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question. Check the maximum limit mentioned in the prompt", "maxLength": 10000, "title": "Response", "type": "string"}, "tagline": {"description": "response created by Vivek", "maxLength": 50, "title": "Tagline", "type": "string"}, "status": {"default": 0, "description": "Status 

In [299]:
messages = PromptTemplate(template="""You are a helpful assistant.Reqspond to the user's query: {query}. 
                        you Must follow these :{format_instruction}\n 
                        IMPORTANT: The max length allowed for the essay is {max_essay_length} CHARACTERS (not words). 
                        Keep your response concise and and well below under this limit unless necessary.
                        The text part of the response (in the response key of json) should be markdown formatted with points and subtopics.""",
                        input_variables=["query"],
                        partial_variables={"format_instruction": resp_parser.get_format_instructions()}
                        )

In [300]:
messages.pretty_print()

You are a helpful assistant.Reqspond to the user's query: {query}. 
                        you Must follow these :The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the LLM model that you (AI) is using", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question. Check the maximum limit mentioned in the prompt", "maxLength": 10000, "title": "Response", "type": "string"}, "tagline": {"description": "response created by Vivek

In [301]:
prompt = messages.format(query="write an essay on misuse of smartphones", max_essay_length=max_essay_length)
prompt

'You are a helpful assistant.Reqspond to the user\'s query: write an essay on misuse of smartphones. \n                        you Must follow these :The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Model response schema.", "properties": {"model_name": {"description": "Name of the LLM model that you (AI) is using", "maxLength": 20, "title": "Model Name", "type": "string"}, "response": {"description": "Response from the model, based on the question. Check the maximum limit mentioned in the prompt", "maxLength": 10000, "title": "Response", "type": "string"}, "tagline": 

In [302]:
# ErrorModelResponse(model_name="gemini-2.0-flash", response=str("hehe"), tagline="response created by Vivek", status=1).model_dump()

In [303]:
# chain = model | resp_parser
# res = chain.invoke(prompt)
model_response = model.invoke(prompt)
print(model_response.content, f"\n\n\n length: {len(model_response.content)}")

# import json
# original_string = model_response.text()
# start_index = original_string.find('{')
# end_index = original_string.rfind('}') + 1

# original_string = original_string[start_index:end_index]
# json_resp = json.loads(original_string)

# json_resp["response"] = json_resp["response"][:max_essay_length]
# model_response.content = json.dumps(json_resp)


try:
    final_response = resp_parser.parse(model_response.content)
except Exception as e:
    print(f"Error parsing response: {e}")
    final_response = ErrorModelResponse(model_name="gemini-2.0-flash", response=str(e), tagline="response created by Vivek", status=1)


```json
{"model_name": "GPT-3", "response": "# The Double-Edged Sword: Misuse of Smartphones\n\nSmartphones, ubiquitous in the 21st century, offer unparalleled connectivity and convenience. However, their misuse presents significant challenges to individuals and society.\n\n*   **Addiction and Mental Health:**\n    *   **Nomophobia:** Fear of being without a smartphone leads to constant checking and anxiety.\n    *   **Social Isolation:** Excessive screen time reduces face-to-face interactions, fostering loneliness and depression.\n    *   **Sleep Disruption:** Blue light emitted from screens interferes with sleep patterns.\n\n*   **Impact on Productivity and Focus:**\n    *   **Distraction:** Notifications and apps constantly interrupt work and studies, decreasing concentration.\n    *   **Procrastination:** Smartphones provide endless entertainment, leading to delayed tasks and decreased efficiency.\n\n*   **Safety Concerns:**\n    *   **Cyberbullying:** Smartphones facilitate online

In [304]:
final_response.model_dump_json()
response_dict = final_response.model_dump()
response_dict

{'model_name': 'GPT-3',
 'response': '# The Double-Edged Sword: Misuse of Smartphones\n\nSmartphones, ubiquitous in the 21st century, offer unparalleled connectivity and convenience. However, their misuse presents significant challenges to individuals and society.\n\n*   **Addiction and Mental Health:**\n    *   **Nomophobia:** Fear of being without a smartphone leads to constant checking and anxiety.\n    *   **Social Isolation:** Excessive screen time reduces face-to-face interactions, fostering loneliness and depression.\n    *   **Sleep Disruption:** Blue light emitted from screens interferes with sleep patterns.\n\n*   **Impact on Productivity and Focus:**\n    *   **Distraction:** Notifications and apps constantly interrupt work and studies, decreasing concentration.\n    *   **Procrastination:** Smartphones provide endless entertainment, leading to delayed tasks and decreased efficiency.\n\n*   **Safety Concerns:**\n    *   **Cyberbullying:** Smartphones facilitate online harass

In [305]:
# Access the 'response' key and store its value in a variable
markdown_string = response_dict['response']
formatted_markdown = markdown_string.replace('\\n', '\n')
print(formatted_markdown)
# The 'markdown_string' variable now holds the desired formatted string.
# You can print it to verify.
# print(markdown_string)

# The Double-Edged Sword: Misuse of Smartphones

Smartphones, ubiquitous in the 21st century, offer unparalleled connectivity and convenience. However, their misuse presents significant challenges to individuals and society.

*   **Addiction and Mental Health:**
    *   **Nomophobia:** Fear of being without a smartphone leads to constant checking and anxiety.
    *   **Social Isolation:** Excessive screen time reduces face-to-face interactions, fostering loneliness and depression.
    *   **Sleep Disruption:** Blue light emitted from screens interferes with sleep patterns.

*   **Impact on Productivity and Focus:**
    *   **Distraction:** Notifications and apps constantly interrupt work and studies, decreasing concentration.
    *   **Procrastination:** Smartphones provide endless entertainment, leading to delayed tasks and decreased efficiency.

*   **Safety Concerns:**
    *   **Cyberbullying:** Smartphones facilitate online harassment and abuse.
    *   **Distracted Driving/Walking

In [306]:
# # When saving to file, Python automatically handles \n as line breaks
# with open('smartphone_misuse.md', 'w', encoding='utf-8') as f:
#     f.write(markdown_string)